In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
import _pickle as pickle
from music21 import instrument, note, stream, chord

In [ ]:
# set to true for debugging
debug = True

In [ ]:
def produce_music():
    
    notes, n_vocab = get_notes('./data/classicalPianoNotes.pkl')
    NetworkInput, mapper = prepare_data(notes, n_vocab)
    model = get_model('./checkpoints/1/ckpt-3.4684-00')
    predictions = predict_notes(model, NetworkInput, mapper, n_vocab)
    create_midi(predictions, 'music.mid')
#-------------------------------------------------------
# uncomment after running all the lower cells.
produce_music()

In [ ]:
def get_notes(notes_file):
    with open(notes_file,'rb') as f :
        notes = pickle.load(f)
    print('Notes are loaded properly using the saved pickle')
    return notes, len(set(notes))
#--------------------------------------------------------
if debug : notes, n_vocab = get_notes('./data/third_piece_MozartNotes.pkl')

In [ ]:
def prepare_data(notes, n_vocab):
    sequence_length = 100
    NetworkInput = []
    mapper = LabelEncoder()
    mapped_notes = mapper.fit_transform(notes)
    
    for i in range(len(notes) - sequence_length):
        in_seq = mapped_notes[i:i+sequence_length]
        NetworkInput.append(in_seq)
    NetworkInput = mapped_notes
    
    n_patterns = len(NetworkInput)
    
    NetworkInput = np.reshape(NetworkInput, (n_patterns, sequence_length, 1))
    NetworkInput = NetworkInput / float(n_vocab)
    
    print('Input shape = ',NetworkInput.shape)
    return NetworkInput, mapper
#--------------------------------------------------
if debug : NetworkInput, mapper = prepare_data(notes, n_vocab2)

In [ ]:
def get_model(saved_path):
    model = load_model(saved_path)
    return model
#--------------------
if debug : model = get_model('./checkpoints/2/ckpt-0.2240-115.hdf5')

In [ ]:
def predict_notes(model, NetworkInput, mapper, n_vocab, length=500):
    start = np.random.randint(0, NetworkInput.shape[0]-1)
    #start = 0
    pattern = NetworkInput.reshape(1,-1,1)
    predictions = []
    
    for i in range(length):
        if i % 100 == 0 : print('.',end='')
        p = model.predict(pattern).argmax()
        predictions.append(mapper.classes_[p])
        
        pattern = np.concatenate(( pattern[0,1:,0], np.asarray([p/float(n_vocab)]) )).reshape(1,-1,1)
    
    print('\nnotes are generated.')
    return predictions
#---------------------------------------------------
if debug : predictions = predict_notes(model, NetworkInput, mapper, n_vocab2)

In [ ]:
def create_midi(predictions, filename):
    offset = 0
    output_notes = []
    
    for pattern in predictions:
        # pattern is a chord
        if '.' in pattern : 
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(current_note)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a single note
        else :
            new_note = note.Note(pattern)
            new_note.storedInstrument = instrument.Piano()
            new_note.offset = offset
            output_notes.append(new_note)
        #increase offset each step
        offset += .5
    
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='./music/generated/'+filename)
    print('midi is generated and saved to disk')
#---------------------------------------------------------------------
if debug : create_midi(predictions, 'thirdpredicted2.mid')